# Freeform Programming Assessment

This notebook uses data from the OpenPowerlifting project, https://www.openpowerlifting.org.

You may download a copy of the data at https://data.openpowerlifting.org or https://openpowerlifting.gitlab.io/opl-csv/bulk-csv.html.

OpenPowerlifting is a community service project to create a permanent, open archive of the world's powerlifting data. All data available on the website is contributed to the public domain.

### Introduction to Powerlifting

Powerlifting is a strength sport comprised of three different disciplines: Squat, Bench Press, and Deadlift. The sum or total of the best lift in each discipline determines the winner.

__Squat:__ A lower-body exercise targeting the quadriceps, glutes, hamstrings, core, and lower back. The lifter holds a barbell across their upper back, and bends their knees into a squatting position until their hips are slightly below parallel. They then return to a standing position.

__Bench Press:__ An upper-body exercise targeting the chest, shoulders, and triceps. Performed lying on a bench, the lifter unracks a barbell at arm's length, and then lowers the bar onto their chest. It is held there momentarily before being pressed back up.

__Deadlift:__ A whole-body exercise primarily engaging the entire posterior chain, and also grip strength. The lifter picks up a 'dead' weight (a loaded barbell) off the floor and assumes a full standing position. 

### Reading in the data

The data will contain powerlifting information for men who participated in a powerlifting meet from the years 2005 - 2024. 

It is a snippet of the full data available on the website, as the file size was too large for Turnitin. Please see 'data_trimming.ipynb' in the project folder to see how I cut down the raw data.

In [ ]:
# importing pandas for data loading and manipulation
import pandas as pd

# reading the compressed CSV file into a pandas dataframe
powerlifting_data = pd.read_csv("male_powerlifting_2005_2024.csv.gz", low_memory = False, compression = "gzip")

# printing the first few rows of the dataframe
powerlifting_data.head()

### Data Analysis

In [ ]:
# checking every column name in the dataframe
print(powerlifting_data.columns)

In [ ]:
# printing shape of the dataframe
print(powerlifting_data.shape)

In [ ]:
# printing summary statistics of the dataframe for squat, bench, deadlift, and total weight lifted
print(powerlifting_data[["Best3SquatKg", "Best3BenchKg", "Best3DeadliftKg", "TotalKg"]].describe())

In [ ]:
# printing the summary information of the dataframe
print(powerlifting_data.info())

In [ ]:
# checking missing values in each column using missingno
import missingno as msno

# this will create a matrix visualization of missing values
msno.matrix(powerlifting_data)

In powerlifting competitions, lifters are given three attempts for each movement (similar to high jump in the Olympics for example). Additionally, sometimes a competitor will make an optional fourth attempt if they want to push for a new personal record (PR). This fourth attempt is not used in scoring.

In 'data_trimming.ipynb', 'Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Squat4Kg', 'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Bench4Kg', 'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg', 'Deadlift4Kg' were all dropped, as they will all contain failed attempts, skipped attempts, or conservative/strategic openers. This saved a lot of file size.

The only value used for rankings and totals is the heaviest successful attempt - this is shown as 'Best3SquatKg', 'Best3BenchKg', and 'Best3DeadliftKg' in the dataset. 

We can also safely drop the 'BirthYearClass', 'Division', 'MeetCountry', 'MeetState', 'MeetTown', 'Federation', 'ParentFederation', and 'Sanctioned' columns, as they will not be needed for this analysis.

In [ ]:
# creating a list of the relevant columns
relevant_columns = ["Name", "Event", "Equipment", "Age", "AgeClass", "BodyweightKg", "WeightClassKg", "Best3SquatKg", "Best3BenchKg", "Best3DeadliftKg", "TotalKg", "Place", "Date", "Year", "Country", "State", "MeetCountry", "MeetName"]

# creating a new dataframe with only the relevant columns
powerlifting_relevant = powerlifting_data[relevant_columns]
powerlifting_relevant.head()

In recent years, powerlifting has seen a surge in popularity.

Can plot a graph of number of unique participants per year to see if the data reflects this.

In [ ]:
# importing matplotlib for plotting and ticker for integer ticks
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# making a copy of the dataframe where individual participants are grouped by year and their unique name
lifters_per_year = powerlifting_relevant.groupby("Year")["Name"].nunique()
# this will output a series with the year as the index and the number of unique lifters as the values

# creating a line plot of lifters per year
plt.figure(figsize = (8, 6))
plt.plot(lifters_per_year.index, lifters_per_year.values, linewidth = 3)
plt.title("Number of Unique Participants Per Year (2005-2024)", fontsize = 16)
plt.xlabel("Year", fontsize = 14)
plt.ylabel("Number of Unique Participants", fontsize = 14)
plt.xlim(2005, 2024)
plt.scatter(2020, lifters_per_year.loc[2020], color = "red", s = 200, zorder = 5)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)
plt.show()

This graph shows participants for all Squat-Bench-Deadlift / SBD powerlifting competitions.

We can clearly see the general trend of a steady increase in participants from 2005-2009, and then a large surge following 2010. 

*Why did popularity increase sharply around 2009?*

- Better data collection and recording of results.
- 'Raw' lifting became widely recognised by many federations, reducing cost and barrier to entry. Expensive gear and equipment was no longer mandatory.
- Many federations expanded and became more standardised - more meets were being held in more cities and countries.
- YouTube and Instagram started to take off - the first fitness videos and 'influencers' started to appear.
- Commercial gyms began installing squat racks (rather than rows of treadmills and bikes).
- While powerlifting isn't an Olympic sport, the 2008 Beijing Olympics likely also increased visibility of strength sports.

Interestingly there is a large drop in participation in 2020. This will almost certainly be due to COVID-19 and lockdowns around the globe - many gyms were closed. Participation didn't drop to zero at this time, as people still held smaller private meets, and many were held online. Lockdown restrictions also differed between countries.

We can plot a choropleth map to see which countries powerlifting is most popular in.

In [ ]:
# choropleth map of unique participants in each country
import plotly.express as px
lifters_per_country = powerlifting_relevant.groupby("Country")["Name"].nunique().reset_index()
fig = px.choropleth(lifters_per_country, locations = "Country", locationmode = "country names", color = "Name", color_continuous_scale = "Viridis", title = "Number of Unique Participants by Country (2005-2024)")
fig.show()

It looks like the USA has way more participants than anywhere else, so signal is hard to see for the rest of the world.

Instead we can plot a barchart to see the top 10 countries where powerlifting is most popular.

In [ ]:
# bar chart of the top 10 countries by number of unique lifters from 2000 to 2025
lifters_per_country = powerlifting_relevant.groupby("Country")["Name"].nunique().sort_values(ascending = False).head(10)
plt.figure(figsize = (10, 6))
plt.bar(lifters_per_country.index, lifters_per_country.values, color = "royalblue")
plt.title("Top 10 Nationalities of Unique Participants (2005-2024)", fontsize = 16)
plt.xlabel("Country", fontsize = 14)
plt.ylabel("Number of Unique Participants", fontsize = 14)
plt.xticks(rotation = 45, fontsize = 10)
plt.yticks(fontsize = 10)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.show()

It looks as though the USA is heavily over-represented in the data.

This looks to be due to a bias in data collection - OpenPowerlifting receives more complete and consistent submissions from US-based federations.

Many participants also have 'NaN' in the Country column:

In [ ]:
# checking number of NaN values in 'Country'
nan_countries = powerlifting_relevant["Country"].isna().sum()
print(f"Number of NaN values in 'Country': {nan_countries}")

That's over 650,000 participations without nationalities recorded.

The MeetCountry column could be a better indicator of how popular powerlifting is across the globe - it is likely more complete.

In [ ]:
# checking the number of NaN values in 'MeetCountry'
nan_meet_countries = powerlifting_relevant["MeetCountry"].isna().sum()
print(f"Number of NaN values in 'MeetCountry': {nan_meet_countries}")

In [ ]:
# plotting a barchart of the top 10 meet countries by number of unique lifters from 2000 to 2025
lifters_per_meet_country = powerlifting_relevant.groupby("MeetCountry")["Name"].nunique().sort_values(ascending = False).head(10)
plt.figure(figsize = (10, 6))
plt.bar(lifters_per_meet_country.index, lifters_per_meet_country.values, color = "seagreen")
plt.title("Top 10 Meet Countries by Unique Participants (2005-2024)", fontsize = 16)
plt.xlabel("Meet Country", fontsize = 14)
plt.ylabel("Number of Unique Participants", fontsize = 14)
plt.xticks(rotation = 45, fontsize = 10)
plt.yticks(fontsize = 10)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.show()

It looks like most competitions recorded in the data are still predominantly US-based.

### Trends in Strength over Time

The next question I wanted to ask was, given the growth in popularity of powerlifting, has overall strength increased over time?

For this question, will look at only SBD events - competitors of these meets will have an SBD total. This will show the combined weight of every lift.

We can also plot the lifts on their own too.

For Best3SquatKg, Best3BenchKg, and Best3DeadliftKg, some values are negative - these need to be removed as they are failed attempts.

In [ ]:
# creating a copy of the dataframe where negative 'Best3SquatKg', 'Best3BenchKg', and 'Best3DeadliftKg' values are filtered out
powerlifting_filtered = powerlifting_relevant[(powerlifting_relevant["Best3SquatKg"] >= 0) & 
                                              (powerlifting_relevant["Best3BenchKg"] >= 0) & 
                                              (powerlifting_relevant["Best3DeadliftKg"] >= 0)].copy()

In [ ]:
# creating axes to get each individual plot side by side
fig, axes = plt.subplots(1, 4, figsize=(20, 5), sharex=True, sharey=True)

# plotting a scatter plot of mean squat, bench, deadlift, and total kg lifted over the years
for i, ax in zip(["Best3SquatKg", "Best3BenchKg", "Best3DeadliftKg", "TotalKg"], axes):
    ax.scatter(powerlifting_filtered["Year"], powerlifting_filtered[i], alpha=0.5)
    ax.set_title(f"{i} Over the Years", fontsize=16)
    ax.set_xlabel("Year", fontsize=14)
    ax.set_ylabel(i, fontsize=14)
    ax.tick_params(axis='x', rotation=45, labelsize=10)
    ax.tick_params(axis='y', labelsize=10)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
plt.show()

These graphs are too messy to get any proper insight from. 

A potentially better graph I could try to make would be the mean total weight lifted each year, for different bodyweight classes. This would remove some of the noise in the data and also the confounding variable of bodyweight.

In [ ]:
# checking value counts in WeightClassKg
print(powerlifting_filtered["WeightClassKg"].value_counts())

I could use the top 5 most common weight classes for the graph.

In [ ]:
# printing the top 5 value counts in WeightClassKg
print(powerlifting_filtered["WeightClassKg"].value_counts().head())

In [ ]:
# checking data type of WeightClassKg
powerlifting_filtered["WeightClassKg"].dtype

In [ ]:
# it is an object data type, which means it is being treated as a string rather than a numeric value
# converting WeightClassKg to floats, and accounting for the '+' sign in some weight classes
powerlifting_filtered["WeightClassKg"] = (powerlifting_filtered["WeightClassKg"].astype(str).str.replace("+", "", regex=False).replace("", "0").astype(float))

In [ ]:
# checking how many people have a WeightClassKg of 0 - these will need to be dropped
zero_weight_class_count = (powerlifting_filtered["WeightClassKg"] == 0).sum()
print(f"Number of participants with WeightClassKg of 0: {zero_weight_class_count}")

In [ ]:
# 119 participants have a WeightClassKg of 0
# dropping these participants from the dataframe
powerlifting_filtered = powerlifting_filtered[powerlifting_filtered["WeightClassKg"] != 0].copy()

# checking value counts in WeightClassKg again to confirm
print(powerlifting_filtered["WeightClassKg"].value_counts())

In [ ]:
# plotting mean totalkg each year for each weight class

# defining the weight classes to plot
weight_classes = [82.5, 83, 90, 93, 100]

plt.figure(figsize=(6, 6))

# looping over each weight class and plotting the mean TotalKg per year
for wc in weight_classes:
    wc_data = powerlifting_filtered[powerlifting_filtered["WeightClassKg"] == wc]
    mean_totalkg_per_year = wc_data.groupby("Year")["TotalKg"].mean()
    
    plt.plot(mean_totalkg_per_year.index, mean_totalkg_per_year.values, label = f'Weight Class {wc}kg', linewidth = 2)
    
plt.title("Mean TotalKg Over the Years by Weight Class", fontsize = 16)
plt.xlabel("Year", fontsize = 14)
plt.ylabel("Mean TotalKg", fontsize = 14)
plt.legend()
plt.xlim(2005, 2024)
plt.ylim(0, 1200)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

It looks like TotalKg lifted has stayed fairly constant over time, with maybe the slightest decrease around 2011.

The 83kg class (yellow) and 93kg class (red) first appear around 2010 as this was when some federations like the IPF started to use them as a new weight class.

### Age and Total Strength Distribution

We can visualise where the data is clustered across age and total weight lifted.

In [ ]:
# hexbin plot of Age vs TotalKg
plt.figure(figsize=(8, 6))
plt.hexbin(powerlifting_filtered["Age"], powerlifting_filtered["TotalKg"], gridsize=30, mincnt=1)
plt.colorbar(label='Count in Bin')
plt.title("Hexbin Plot of Age and TotalKg", fontsize=16)
plt.xlabel("Age", fontsize=14)
plt.ylabel("TotalKg", fontsize=14)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

The dataset is highly populated by lifters aged roughly 18-25, and it looks like they generally perform in the 450-600kg total range.

### Correlations between Bodyweights and Strength

You would probably expect heavier participants to be able to lift heavier weights. 

We can investigate if there is a link between bodyweight and strength by plotting the mean weight for each lift against bodyweight.

In [ ]:
# looping over best3s and plotting means in scatter plot against weight class
for lift in ["Best3SquatKg", "Best3BenchKg", "Best3DeadliftKg", "TotalKg"]:
    plt.figure(figsize=(8, 6))
    mean_lift_by_weight_class = powerlifting_filtered.groupby("WeightClassKg")[lift].mean()
    plt.scatter(mean_lift_by_weight_class.index, mean_lift_by_weight_class.values, alpha=0.7)
    plt.title(f"Mean {lift} by Weight Class", fontsize=16)
    plt.xlabel("Weight Class (Kg)", fontsize=14)
    plt.ylabel(f"Mean {lift} (Kg)", fontsize=14)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.show()


It looks like there is a strong positive correlation between weight class and the mean kgs lifted for each lift.

The relationship doesn't look completely linear - can compare R2's of linear regression line and sigmoidal line.

In [ ]:
import seaborn as sns
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit

# defining the sigmoidal function
# L: amplitude, x0: midpoint, k: steepness, b: baseline
def sigmoid(x, L, x0, k, b):
    return L / (1 + np.exp(-k * (x - x0))) + b

# looping over best3s + total and plotting means in scatter plot against weight class with linear and sigmoidal fits   
for lift in ["Best3SquatKg", "Best3BenchKg", "Best3DeadliftKg", "TotalKg"]:
    fig, ax = plt.subplots(figsize = (8, 6))

    # aggregate and sort 
    mean_df = (powerlifting_filtered.groupby("WeightClassKg")[lift].mean().reset_index().sort_values("WeightClassKg"))

    # converting to numpy arrays for fitting
    x = mean_df["WeightClassKg"].to_numpy()
    y = mean_df[lift].to_numpy()

    # making scatter plot of the data points
    sns.scatterplot(data = mean_df, x = "WeightClassKg", y = lift, alpha = 0.7, s = 50, color = "gray", edgecolor = "none", ax = ax, label = "Data")

    # linear fit + R2 
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    r2_linear = r_value**2
    
    # draw linear line 
    x_smooth = np.linspace(x.min(), x.max(), 400)
    y_lin = slope * x_smooth + intercept
    ax.plot(x_smooth, y_lin, color = "royalblue", linewidth = 3, label = f"Linear: R2 = {r2_linear:.3f}")

    # sigmoidal fit + R2
    try:
        # sensible initial guesses
        p0 = [y.max() - y.min(), np.median(x), 0.1, y.min()]
        popt, _ = curve_fit(sigmoid, x, y, p0 = p0, maxfev = 20000)
        y_sig = sigmoid(x, *popt)
        r2_sig = 1 - np.sum((y - y_sig)**2) / np.sum((y - y.mean())**2)

        x_smooth = np.linspace(x.min(), x.max(), 400)
        ax.plot(x_smooth, sigmoid(x_smooth, *popt),
                color = "darkorange", linewidth = 3,
                label = f"Sigmoidal: R2 = {r2_sig:.3f}")
    except Exception:
        # If the fit fails, skip sigmoid
        pass

    ax.set_title(f"Mean {lift} by Weight Class", fontsize = 16)
    ax.set_xlabel("Weight Class (Kg)", fontsize = 14)
    ax.set_ylabel(f"Mean {lift} (Kg)", fontsize = 14)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.tick_params(labelsize = 10)
    ax.legend(fontsize = 12)
    plt.show()

The R^2 value is higher for the sigmoidal curve than for the linear line in all four graphs.

This suggests that the relationship between weight and mean squat, bench, deadlift, and total lifted is non-linear, and that the sigmoidal model explains a greater proportion of variation than a linear model.

This could suggest that past ~100kg bodyweight, there are diminishing returns for strength gains. It is possible that on average, this is approaching the human limit, although the presence of outlier groups indicates a large amount of individual variability.

Many of the lighter bodyweight lifters are younger, so there could potentially be an age and 'training age' effect going on too - lighter weight classes might disproportionately include younger or less experienced lifters, whereas heavier classes may be more older and/or experienced lifters. 

This could also tie into the 'newbie gains' concept in lifting, where strength improvements generally plateau the longer an individual has been training. However as the dataset does not contain training age, this is just a hypothesis.

### Correlations between each Lift

We can see how squat, bench, and deadlift co-vary between lifters within each weight class.

We would likely expect to see that all lifts are correlated similar amounts for each weight class.

In [ ]:
# writing a function that takes the dataframe and weight class as inputs and returns a correlation matrix plot with best squat, bench, deadlift, and total kg

# importing seaborn for heatmap plotting
import seaborn as sns

# defining the function
def correlation_matrix_by_weight_class(df, weight_class):
    
    # filtering the dataframe for the specified weight class
    wc_data = df[df["WeightClassKg"] == weight_class]

    # checking if the weight class exists in the dataframe
    if wc_data.empty:
        print(f"No data available for weight class {weight_class}kg.")
        return
    
    # selecting only the relevant columns for correlation
    corr_data = wc_data[["Best3SquatKg", "Best3BenchKg", "Best3DeadliftKg", "TotalKg"]].dropna()

    # checking if there are enough data points to calculate correlation
    if len(corr_data) < 2:
        print(f"Not enough data to calculate correlation for weight class {weight_class}kg.")
        return
    
    # calculating the correlation matrix
    corr_matrix = corr_data.corr()
    
    # plotting the correlation matrix
    plt.figure(figsize = (8, 6))
    sns.heatmap(corr_matrix, annot = True, cmap = 'coolwarm', vmin = -1, vmax = 1)
    plt.title(f'Correlation Matrix for Weight Class {weight_class}kg', fontsize = 16)
    plt.show()

In [ ]:
# using the function to plot the correlation matrix for the 83kg weight class
correlation_matrix_by_weight_class(powerlifting_filtered, 83)

In [ ]:
# using the function to plot the correlation matrix for the 100kg weight class
correlation_matrix_by_weight_class(powerlifting_filtered, 100)

The correlations between each lift stay largely similar between bodyweight groups. However, there will be a number of confounding variables in each participant and lift.

Interestingly, bench and deadlift have the weakest correlation. 

This could be due to leverages and range of motion (ROM) - for example, arm length plays a large role in both of these lifts. Longer arms are useful for deadlifts, as you would bend down less to reach the bar on the floor - this would be a shorter ROM. However this is less beneficial for bench press, where longer arms would mean a longer distance for the bar to travel down and be pressed back up - a larger ROM.